In [1]:
import datajoint as dj
import numpy as np
import time
from tqdm import tqdm

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 200

ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema('microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [3]:
@schema
class CoarseLabel(dj.Computed):
    definition = """
    # Vertex labels for ta3p100.ProofreadLabel did not correctly match the triangle labels, so these are regenerated from the correct triangle labels.
    -> ta3p100.ProofreadLabel
    ---
    vertices  : longblob # Corrected vertex labels
    triangles : longblob # Same triangle labels as ta3p100.ProofreadLabel
    """
    
    key_source = ta3p100.ProofreadLabel & 'status="complete"'
    
    def make(self, key):
        start = time.time()
        
        print(key["segment_id"])
        labels = (ta3p100.ProofreadLabel & key).fetch1()
        corrected_vertex_labels = np.zeros(labels['vertices'].shape, np.uint8)
        
        mesh = (ta3p100.CleansedMesh & 'decimation_ratio=0.35' & dict(segment_id=key['segment_id'])).fetch1()
        mesh_triangles = mesh['triangles']
        
        vertex_label_dict = dict()
        for i, label in enumerate(labels['triangles']):
            triangle = mesh_triangles[i]
            for node in triangle:
                if node in vertex_label_dict:
                    if vertex_label_dict[node] < label:
                        vertex_label_dict[node] = label
                else:
                    vertex_label_dict[node] = label
                
        for node, label in vertex_label_dict.items():
            corrected_vertex_labels[node] = label
            
        self.insert1(dict(key,
                          vertices=corrected_vertex_labels,
                          triangles=labels['triangles']))
        
        print("Segment {} vertex labels regenerated in: {} seconds.".format(key['segment_id'], time.time() - start))

In [4]:
start = time.time()
CoarseLabel.populate(reserve_jobs=True)
print(time.time() - start)

648518346349471910
Segment 648518346349471910 vertex labels regenerated in: 6.974725246429443 seconds.
648518346349472601
Segment 648518346349472601 vertex labels regenerated in: 4.906996965408325 seconds.
648518346349473160
Segment 648518346349473160 vertex labels regenerated in: 5.860327243804932 seconds.
648518346349473597
Segment 648518346349473597 vertex labels regenerated in: 4.705285310745239 seconds.
648518346349473804
Segment 648518346349473804 vertex labels regenerated in: 5.496031045913696 seconds.
648518346349473813
Segment 648518346349473813 vertex labels regenerated in: 4.579697132110596 seconds.
648518346349473816
Segment 648518346349473816 vertex labels regenerated in: 4.370855808258057 seconds.
648518346349473822
Segment 648518346349473822 vertex labels regenerated in: 2.434675931930542 seconds.
648518346349473830
Segment 648518346349473830 vertex labels regenerated in: 2.660757064819336 seconds.
648518346349473833
Segment 648518346349473833 vertex labels regenerated i

648518346349501787
Segment 648518346349501787 vertex labels regenerated in: 4.767602205276489 seconds.
648518346349502434
Segment 648518346349502434 vertex labels regenerated in: 3.6439380645751953 seconds.
648518346349502767
Segment 648518346349502767 vertex labels regenerated in: 5.644468784332275 seconds.
648518346349503140
Segment 648518346349503140 vertex labels regenerated in: 6.305984973907471 seconds.
648518346349503588
Segment 648518346349503588 vertex labels regenerated in: 4.5345118045806885 seconds.
648518346349503643
Segment 648518346349503643 vertex labels regenerated in: 8.553354024887085 seconds.
648518346349503767
Segment 648518346349503767 vertex labels regenerated in: 7.656324148178101 seconds.
648518346349504130
Segment 648518346349504130 vertex labels regenerated in: 11.953754186630249 seconds.
648518346349504324
Segment 648518346349504324 vertex labels regenerated in: 15.958139896392822 seconds.
648518346349504565
Segment 648518346349504565 vertex labels regenerat

In [5]:
#ta3p100.CoarseLabel()

In [6]:
#(schema.jobs & "table_name='__coarse_label'").delete()